In [ ]:
#!/usr/bin/python 
# -*- coding: utf-8 -*-

In [ ]:
def get_info(res, get_content=True):
    '''
    获取指定的内容(题目和评论)或问题id
    '''
    from bs4 import BeautifulSoup  # 对html格式的文件, 处理
    from time import sleep
    
    # 考虑申请失败的情况
    if res.status_code != 200:
        print("    ip被封!, 等候10s")
        sleep(10)
        if get_content:
            return 0
        else:
            return []
    
    # 正常处理
    contents_dict = res.json()["data"]
    if get_content:
        contents_list = []
        for item in contents_dict:
            temp_dict = {}
            
            temp_dict["title"] = item["question"]["title"]
            
            content = ""
            html = item["content"]
            bf = BeautifulSoup(html)
            for p in bf.stripped_strings:
                content += p
            temp_dict["describe"] = content
            
            contents_list.append(temp_dict)
        return contents_list
    else:
        question_id_list = []
        for item in contents_dict:
            content = ""
            question_id = str(item["id"])
            question_id_list.append(question_id)
        return question_id_list

In [ ]:
def push_es(content_list):
    '''
    将数据推入云es库中
    '''
    # 考虑列表为空时, 直接跳出
    if len(content_list)==0:
        return
    
    # 列表非空时, 开始推入
    from elasticsearch import Elasticsearch
    from elasticsearch import helpers
    # 开始连接es库(感觉这里可以优化)
    es = Elasticsearch(
        ['es-cn-09k1xdlqf000a7yyn.public.elasticsearch.aliyuncs.com'],
        http_auth=('elastic', 'Zy13720028637'),
        port=9200,
        use_ssl=False
    )
    # 将数据修饰一下
    action = ({
        "_index":"zhihu",
        "_type":"question",
        "_source":data
    } for data in content_list)
    # 导入数据
    helpers.bulk(es, action)

In [ ]:
def grap_web(question_id):
    '''
    用于获取该问题号对应的所有评论
    '''
    import requests
    from time import sleep
    # 爬取网站, num控制每次爬取评论数目
    num = 5
    offset = 0 # 起始值
    i = 0
    ## 准备爬取的网站
    base_url = "https://www.zhihu.com/api/v4/questions/" + question_id + "/answers?include=data%5B*%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_labeled%2Cis_recognized%2Cpaid_info%2Cpaid_info_content%3Bdata%5B*%5D.mark_infos%5B*%5D.url%3Bdata%5B*%5D.author.follower_count%2Cbadge%5B*%5D.topics%3Bsettings.table_of_content.enabled%3B&limit={}&offset={}&sort_by=default&platform=desktop"
    print("\n  抓取知乎问题号{}:".format(question_id))
    while True:
        url = base_url.format(num, offset + i*num)
        ## 准备https头部
        headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"}
        # ## 睡眠
        sleep(0.75)
        ## 开始访问
        res = requests.get(url, headers=headers)
        ## 处理json数据, 获取回答和题目
        contents_list = get_info(res)
        ## 对于IP被封或请求失败时的处理
        if contents_list==0:
            print("    再次申请...")
            continue
        ## 存储数据到es中
        push_es(contents_list)
        ## 判断截至条件
        if len(contents_list) < num:
            if len(contents_list) != 0:
                print("    已抓取{}条评论!".format(offset + i*num + len(contents_list)))
            print("  问题号:" + question_id +"抓取完成!")
            break
        i += 1
        print("    已抓取{}条评论!".format(offset + i*num))

In [ ]:
def to_more_grap(question_id):
    '''
    用于获取相关问题的问题id
    '''
    from time import sleep
    import requests
    # 准备网站
    base_url = "https://www.zhihu.com/api/v4/questions/" + question_id + "/similar-questions?include=data%5B*%5D.answer_count%2Cauthor%2Cfollower_count&limit=5"
    
    ## 准备https头部
    headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"}
    # ## 睡眠
    sleep(0.75)
    ## 开始访问
    res = requests.get(base_url, headers=headers)
    ## 处理json数据, 获取回答和题目
    contents_list = get_info(res, get_content=False)
    return contents_list

In [ ]:
def set_grap(question_id, question_num=1):
    '''
    集成爬虫
    '''
    # 爬取网站
    grap_web(question_id)
    question_num -= 1

    question_id = [question_id]
    i = 1
    while question_num:
        temp_i = i
        while len(question_id) == i:
            question_id += to_more_grap(question_id[temp_i-1])
            question_id = sorted(set(question_id),key=question_id.index)
            temp_i -= 1
            if temp_i == -1:
                print("\n因问题一致, 故抓取结束!")
                return
        grap_web(question_id[i])
        i += 1
        question_num -= 1
    print("\n抓取结束!")

In [ ]:
def zhihu_grap(num=5):
    # 默认id :339505719, 48510028, 20399824, 365863994, 264813121
    print("输入抓取知乎对应问题的号码:")
    question_id = input()
    print("输入成功!")
    set_grap(question_id, question_num=num)

In [ ]:
def es_search(search_in):
    '''
    根据search_in(指定字符串)查找相关问题, 并返回全部内容
    '''
    from elasticsearch import Elasticsearch
    # 开始连接es库(感觉这里可以优化)
    es = Elasticsearch(
        ['es-cn-09k1xdlqf000a7yyn.public.elasticsearch.aliyuncs.com'],
        http_auth=('elastic', 'Zy13720028637'),
        port=9200,
        use_ssl=False
    )
    
    # 开始查询(这里绝对可以优化, 先简单实现)
    dsl = {
        'query': {
            'match': {
                'title': search_in
            }
        }
    }

    return es.search(index='zhihu', doc_type='question', body=dsl)

In [ ]:
def get_search_info(content):
    '''
    先简单一点, 就获取相关问题title
    '''
    title_list = []
    for data in content["hits"]["hits"]:
        if len(title_list)>=5:
            break
        source = data["_source"]
        title = source["title"]
        if title not in title_list:
            title_list.append(title)
    return title_list

In [ ]:
def info_print(content_info):
    '''
    简单实现, 就打印含有title的列表
    '''
    print("  搜索结果:")
    for index, title in enumerate(content_info):
        print("    {}. {}".format(index+1, title))

In [ ]:
def zhihu_search():
    # 输入相关查询
    search_in = input()
    # 从es库查询结果
    content = es_search(search_in)
    # 处理信息, 获取主要内容
    content_info = get_search_info(content)
    # 打印内容
    info_print(content_info)

In [ ]:
if __name__=="__main__":
    while True:
        print("------执行菜单------")
        print("1. 简单抓取")
        print("2. 相关搜索")
        print("3. 退出程序")
        select_num = int(input())
        if select_num==1:
            zhihu_grap()
        elif select_num==2:
            zhihu_search()
        else:
            print("成功退出!")
            break

------执行菜单------
1. 简单抓取
2. 相关搜索
3. 退出程序
输入抓取知乎对应问题的号码:
输入成功!

  抓取知乎问题号264813121:
    已抓取5条评论!
    已抓取6条评论!
  问题号:264813121抓取完成!

  抓取知乎问题号429366047:
    已抓取5条评论!
    已抓取9条评论!
  问题号:429366047抓取完成!

  抓取知乎问题号427305314:
    已抓取5条评论!
    已抓取10条评论!
    已抓取15条评论!
    已抓取20条评论!
    已抓取25条评论!
    已抓取30条评论!
    已抓取35条评论!
    已抓取40条评论!
    已抓取41条评论!
  问题号:427305314抓取完成!

  抓取知乎问题号265072222:
    已抓取5条评论!
    已抓取10条评论!
    已抓取12条评论!
  问题号:265072222抓取完成!

  抓取知乎问题号269618813:
    已抓取5条评论!
    已抓取10条评论!
    已抓取11条评论!
  问题号:269618813抓取完成!

抓取结束!
------执行菜单------
1. 简单抓取
2. 相关搜索
3. 退出程序
  搜索结果:
------执行菜单------
1. 简单抓取
2. 相关搜索
3. 退出程序
  搜索结果:
    1. 有没有靠谱的母婴号推荐？
    2. 有哪些比较好的母婴公众号推荐？
    3. 宝宝刚出生大家有什么母婴好物推荐吗？？ ?
------执行菜单------
1. 简单抓取
2. 相关搜索
3. 退出程序
  搜索结果:
------执行菜单------
1. 简单抓取
2. 相关搜索
3. 退出程序
  搜索结果:
    1. 宝宝刚出生大家有什么母婴好物推荐吗？？ ?
------执行菜单------
1. 简单抓取
2. 相关搜索
3. 退出程序
  搜索结果:
    1. 宝宝刚出生大家有什么母婴好物推荐吗？？ ?
------执行菜单------
1. 简单抓取
2. 相关搜索
3. 退出程序
成功退出!
